# DeepL
- DeepL API을 활용해서 번역을 하는 API 이다.
- 대부분의 오픈소스 LLM이 영어가 한글보다 훨씬 잘되기 때문에 영어로 질문하고 답을 한글로 번역하기 좋다고 한다.

In [18]:
import deepl
from dotenv import load_dotenv
import os

load_dotenv()

True

In [19]:
translator = deepl.Translator(os.getenv("DEEPL_API_KEY"))
result = translator.translate_text("Hello, world!", target_lang="Ru")
print(result.text)

Привет, мир!


### OpenAI API 형식으로 질문합니다.

In [20]:
from openai import OpenAI

question = "What is the capital of the South Korea?"

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a friendly AI."},
        {"role": "user", "content": question},
    ],
    temperature=0.5,
)

print(completion.choices[0].message.content)
print(completion.choices)
print(completion.choices[0].message)

The capital of South Korea is Seoul.
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of South Korea is Seoul.', role='assistant', function_call=None, tool_calls=None))]
ChatCompletionMessage(content='The capital of South Korea is Seoul.', role='assistant', function_call=None, tool_calls=None)


### 같은 Openai API 형식으로 Llama3 8B를 내 PC에서 실행해보기
- OpenAI API 형식은 간단한 chat message를 만들기에 좋다. 많은 LLM들이 이런 형식을 따르고 있다. (LM Studio 참고)
- Llama3 8B는 성능이 좋다. 그러나 한국어 답변 안됨. 한국어로 fine tuning한 모델들은 성능이 떨어진다.
- 따라서, fine tuning을 열심히 하는 것 보다는 한국어-영어를 번역하는 편이 좋을 경우가 많다고함.
- LM Studio에서 local server를 실행한 후 다음을 실행해야함.

In [21]:
from openai import OpenAI

question = "What is the capital of North Korea?"

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
    model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
    messages=[
        {
            "role": "system",
            "content": "Please Answer the follwing question like professional.",
        },
        {"role": "user", "content": question},
    ],
    temperature=0.7,
)

print(completion.choices[0].message)
print()
print(completion.choices[0].message.content)

ChatCompletionMessage(content='The capital of North Korea is **Pyongyang**.  \n', role='assistant', function_call=None, tool_calls=None)

The capital of North Korea is **Pyongyang**.  



In [22]:
answer = translator.translate_text(
    completion.choices[0].message.content, target_lang="KO"
).text
print(answer)

북한의 수도는 **평양** 입니다.  



### gemma2사용 : 한국어 가능

In [23]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
question = "북한에 대해 알려줘."

completion = client.chat.completions.create(
    model="bartowski/gemma-2-9b-it-GGUF",
    messages=[
        {"role": "system", "content": "Always answer professionally."},
        {"role": "user", "content": question},
    ],
    temperature=0.7,
)


answer = completion.choices[0].message.content
print(answer)

KeyboardInterrupt: 

### Ai assistant 모드
방금껀 chat 모드

In [25]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {
        "role": "system",
        "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful.",
    },
    {"role": "user", "content": "북한의 수도는 어디야?"},
]

while True:
    completion = client.chat.completions.create(
        model="bartowski/gemma-2-9b-it-GGUF",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}

    print("Assistant: ", end="", flush=True)
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content
    history.append(new_message)
    print()
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "종료"]:
        break
    print("User: ", user_input)
    history.append({"role": "user", "content": user_input})

Assistant: 북한의 수도는 **평양**입니다.  

User:  북한의 지도자는?
Assistant: 북한의 현재 지도자는 **김정은** 입니다. 

그는 2011년 김정일 국방위원장 사망 이후 북한의 최고 지도자가 되었습니다.  

User:  북한과 한국의 차이점은?
Assistant: 북한과 한국은 역사적, 정치적으로 많은 차이를 보입니다. 핵심적인 차이점은 다음과 같습니다:

**1. 정치 체제:**

* **남한:**  서구식 민주주의 체제를 가지고 있으며, 대통령을 중심으로 하는 삼부분 분립(입법부, 집행부, 사법부)의 행정 구조를 갖추고 있습니다.
* **북한:** 공산주의 체제이며,  "김일성-김정일사상"에 기반한 강력한 한당이 통치합니다. 권력은 김정은을 중심으로 집중되어 있으며, 입법부는 국가의 의도를 수행하는 역할을 합니다.

**2. 경제 체제:**

* **남한:** 자유 시장 경제를 기반으로 하며 민간 기업 활동이 발달했습니다.  
* **북한:** 계획경제 체제로, 정부가 경제의 모든 측면을 통제합니다. 중앙 집권적이며, 국제 무역과 외국인 투자는 제한되어 있습니다.

**3. 사회 문화:**

* **남한:**  다양한 종교와 사상이 존재하며, 개방적인 사회 문화를 가지고 있습니다. 인터넷 접근성도 높습니다.
* **북한:** 국가에서 통제하는 유일한 공식 교리인 "김일성-김정일사상"을 강조하며, 다른 이념이나 종교는 금지됩니다.  정보 접근이 제한되어 있으며, 외국과의 교류가 매우 제한적입니다.

**4. 국제 관계:**

* **남한:**   주로 미국, 일본, 유럽 국가들과 외교 관계를 맺으며, 동북아시아 지역 내에서도 중요한 역할을 하고 있습니다.
* **북한:**  국제 사회와의 관계가 매우 복잡하며, 미국의 주요 적이며 군사적 위협으로 인식됩니다.

이러한 차이는 북한과 한국 간의 심각한 분열을 만들어냈습니다.



User:  너는 북한에 대해 어떻게 생각해? 짧게 말하렴
Assistant: 북한은 고립되고, 국

BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}